<a href="https://colab.research.google.com/github/daviddkovacs/Granger_Causality_global/blob/main/Granger_causality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing libraries and accessing GEE

Please, before setting up the GEE REST API framework, follow the instructions on: [https://github.com/KMarkert/restee](https://)

In [ ]:
# Please provide your Google Project ID
PROJECTID = ''

In [ ]:
import ee
from google.colab import drive
import os
import restee as ree
from osgeo import osr
from osgeo import gdal
from google.auth.transport.requests import AuthorizedSession
from statsmodels.tsa.stattools import grangercausalitytests
import ee
import numpy as np
import time
import matplotlib.pyplot as plt
from ipywidgets import Text
import pandas as pd
import datetime
import cv2
from multiprocessing import Pool, Process
import itertools
import time
import dask
import random
import dask.dataframe as dd
from dask.distributed import Client, progress
import numpy.ma as ma
import xarray as xr

drive.mount('/content/drive')
!gcloud auth login --project {PROJECTID}
ee.Authenticate()
!pip install restee
!pip install xesmf
!pip install wxee
!pip install rasterio

session = AuthorizedSession(ee.data.get_persistent_credentials())

class EESessionContainer(ree.EESession):
    def __init__(self, project, session):
        self._PROJECT = project
        self._SESSION = session
ee_session = EESessionContainer(PROJECTID, session)
ee.Initialize(ee_session.session.credentials, project=PROJECTID)

Download and insert the REST API json key and connect the Colab notebook to GEE

In [ ]:
session = ree.EESession(PROJECTID,"insert REST API json key here")
np.set_printoptions(suppress=True)
ee.Initialize()

# Setting up spatio-temporal data for ELV and VP

In the next cell, you can select what environmental variable you want to compare
Use any ERA 5 Land variable from: [https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_DAILY_RAW](https://)


You can further select the vegetation product of causality analysis in the next cell, line 73

In [1]:
ERA5Tvar = "ERA 5 Land variable"

In [ ]:
def add_datumSIF(x):
    x = x.set({'system:time_start': ee.Date(x.getString("system:index").slice(0,4)
    .cat("-")
    .cat(x.getString("system:index").slice(5,7))
    .cat("-")
    .cat(x.getString("system:index").slice(8,10))).millis()})
    return x


def add_datumSIF2019(x):
    datum = x.getString("system:index").slice(13,25)
    return x.set({'system:time_start': ee.Date(datum)})

waterMask = ee.Image('CIESIN/GPWv411/GPW_Water_Mask').select('water_mask').eq(3);
bareMask = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('bare-coverfraction').lte(90).eq(0);


def masking(x):
    return x.mask(waterMask)

SIF = ee.ImageCollection("projects/ee-dkvcsdvd/assets/SIF/TROPOSIF1822").map(add_datumSIF)


startDateStr = '2020-01-01'
endDateStr = '2022-01-01'

# MODISvar = "Lai"
startDateGEE = ee.Date(startDateStr)
endDate = datetime.datetime.strptime(endDateStr, '%Y-%m-%d').date()
timeWindows = 10
startDate = datetime.datetime.strptime(startDateStr, "%Y-%m-%d").date()
daysIterations = abs((startDate - endDate)//timeWindows).days

coordinates = [[[-76.99349151087304, 67.1700297446974],
          [-76.99349151087304, -43.89775352375589],
          [104.412758489127, -43.89775352375589],
          [104.412758489127, 67.1700297446974]]]
fc = ee.FeatureCollection("projects/ee-dkvcsdvd/assets/Output/World_shapefile")
area = fc.geometry()

domain = ree.Domain.from_ee_geometry(session,area,resolution=0.5)

stack = np.array([])
stack2 = np.array([])

for i in range(0,daysIterations):
    try:
        fecha_inicio = startDateGEE.advance(ee.Number(i).multiply(timeWindows),'day')
        fecha_fin = fecha_inicio.advance(timeWindows, 'day')
        ERA5T = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_RAW").filterDate(fecha_inicio,fecha_fin).select(ERA5Tvar)
        LST = ee.ImageCollection('MODIS/061/MOD11A1').filterDate(fecha_inicio,fecha_fin).select("LST_Day_1km")
        SM = ee.ImageCollection("NASA/SMAP/SPL3SMP_E/005").filterDate(fecha_inicio,fecha_fin).select("soil_moisture_am")


        def add_datumSIF(x):
            x = x.set({'system:time_start': ee.Date(x.getString("system:index").slice(0,4)
            .cat("-")
            .cat(x.getString("system:index").slice(5,7))
            .cat("-")
            .cat(x.getString("system:index").slice(8,10))).millis()})
            return x

        def add_datumCGLS(x):
            x = x.set({'system:time_start': ee.Date(x.getString("system:index").slice(0,4)
            .cat("-")
            .cat(x.getString("system:index").slice(4,6))
            .cat("-")
            .cat(x.getString("system:index").slice(6,9))).millis()})
            return x


# The following line selects the vegetation product you want to analyse, right now FVC is used
        cgls_biopar = ee.ImageCollection("projects/ee-dkvcsdvd/assets/CGLS/CGLS_FVC_20_22_300m").map(add_datumCGLS)
        cgls_biopar = cgls_biopar.filterDate(fecha_inicio,fecha_fin).select("b1")

# For TROPOSIF dataa
        SIF = ee.ImageCollection("projects/ee-dkvcsdvd/assets/SIF/TROPOSIF1822").map(add_datumSIF)
        SIF = SIF.filterDate(fecha_inicio,fecha_fin).select("b1")

        fecha_str = datetime.datetime.utcfromtimestamp(fecha_inicio.getInfo()['value']/1000.0).strftime('%Y%m%d')
        print(fecha_str)


        waterMask_array = ree.img_to_xarray(session,domain, waterMask,no_data_value=np.nan)
        waterMask_array = waterMask_array.to_array().values[0, :, :]
        bareMask_array = ree.img_to_xarray(session,domain, bareMask,no_data_value=np.nan)
        bareMask_array = bareMask_array.to_array().values[0, :, :]


        result = cgls_biopar.mean()

        gee_array = ree.img_to_xarray(session,domain, result,no_data_value=np.nan)
        gee_array = gee_array.to_array().values[0, :, :]


        gee_array = np.ma.masked_array(gee_array,waterMask_array)

        gee_array = np.ma.masked_array(gee_array,bareMask_array)

        print("gee_array")
        plt.figure(figsize = (25,15))
        plt.imshow(gee_array)
        plt.colorbar()
        plt.show()

        result2 = ERA5T.mean().divide(86400).setDefaultProjection(ERA5T.first().projection())

        gee_array2 = ree.img_to_xarray(session,domain, result2,no_data_value=np.nan)
        gee_array2 = gee_array2.to_array().values[0, :, :]

        gee_array2 = np.ma.masked_array(gee_array2,waterMask_array)
        gee_array2 = np.ma.masked_array(gee_array2,bareMask_array)

        print("gee_array2")
        plt.figure(figsize = (25,15))
        plt.imshow(gee_array2)
        plt.colorbar()
        plt.show()
        stack2 = np.dstack([stack2, gee_array2]) if stack2.size else gee_array2

        stack = np.dstack([stack, gee_array]) if stack.size else gee_array
    except:
        pass

#Granger Causality algorithm

In the next cell, select the lag you want the VAR model to use.
Additionall, set Ftest_Pval to 0 to obtain F statistic maps and 1 to obtain P value maps

In [ ]:
lag = 10
Ftest_Pval = # Use 0 for F test and 1 for P val

In [ ]:
xdim, ydim, num_images = stack.shape
granger_array = np.zeros(shape=(xdim, ydim))
output_array = np.empty((xdim, ydim))

for i in range(xdim):
    for j in range(ydim):
        try:
            ts_1 = []
            ts_2 = []
            for im_n in range(num_images):
                ts_1.append(stack[i][j][im_n])
                ts_2.append(stack2[i][j][im_n])
            df = pd.DataFrame(list(zip(ts_1, ts_2)), columns=['ts_1', 'ts_2'])
            df = df.diff().iloc[1:]
            lags = lag
            maxlag = lags
            test = 'params_ftest'
            ForP = Ftest_Pval # 0:Ftest 1:p-value
            gc_res = grangercausalitytests(df, lags,verbose = False)[lags][0][test][ForP]
            output_array[i, j] = gc_res
        except:
            pass

#Visualizing the Granger Causality map

In [ ]:
world = output_array
waterMask_array = ree.img_to_xarray(session,domain, waterMask,no_data_value=np.nan)
waterMask_array = waterMask_array.to_array().values[0, :, :]
bareMask_array = ree.img_to_xarray(session,domain, bareMask,no_data_value=np.nan)
bareMask_array = bareMask_array.to_array().values[0, :, :]
output_array = np.ma.masked_array(output_array,waterMask_array)
output_array = np.ma.masked_array(output_array,bareMask_array)

plt.figure(figsize = (25,10))
plt.imshow(output_array,vmin=0, vmax=1)
plt.colorbar()
plt.show()